In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import numba

In [ ]:
import numpy as np
import numba

def getPositionsCharges(boxSize: int, charge: float = 1.0):
    """
    Generate positions and charges for a 3D lattice of alternating charges.

    Parameters:
    - boxSize (int): The size of the cubic box (number of lattice points along each axis).
    - charge (float, optional): The magnitude of the charge. Default is 1.0.

    Returns:
    - positions (np.ndarray): An array of shape (N, 3) containing the positions of the charges.
    - charges (np.ndarray): An array of shape (N,) containing the charges at each position.
    """
    # Generate a grid of integer coordinates within the box
    mgrid = np.mgrid[0:boxSize, 0:boxSize, 0:boxSize].reshape(3, -1).T

    # Shift positions by 0.5 to center them within the unit cell
    positions = mgrid + 0.5

    # Assign alternating charges based on the sum of the indices
    charges = charge * (-1) ** (mgrid.sum(axis=1))

    return positions.astype(np.float64), charges.astype(np.float64)

# Precomputed constants for the erfc approximation
PA = 3.97886080735226

erfcCoeffs = np.array([
    2.75374741597376782e-1,
    4.90165080585318424e-1,
    7.74368199119538609e-1,
    1.07925515155856677,
    1.31314653831023098,
    1.37040217682338167,
    1.18902982909273333,
    0.805276408752910567,
    0.357524274449531043,
    0.0166207924969367356,
    -0.119463959964325415,
    -0.0838864557023001992,
    0.00249367200053503304,
    0.0390976845588484035,
    0.0161315329733252248,
    -0.0133823644533460069,
    -0.0127223813782122755,
    0.00383335126264887303,
    0.00773672528313526668,
    -0.000870779635317295828,
    -0.00396385097360513500,
    0.000119314022838340944,
    0.00127109764952614092
])

@numba.vectorize([numba.float64(numba.float64)])
def erfc(x):
    """
    Compute the complementary error function using a polynomial approximation.

    Parameters:
    - x (float): The input value.

    Returns:
    - y (float): The value of erfc(x).
    """
    # Transformation variable
    t = PA / (PA + np.abs(x))
    u = t - 0.5

    # Compute powers of u
    uPowers = u ** np.arange(23)

    # Evaluate the polynomial approximation
    y = np.sum(uPowers * erfcCoeffs)

    # Multiply by the exponential factor
    y *= t * np.exp(-x**2)

    return y

@numba.vectorize([numba.float64(numba.float64, numba.float64)])
def wrap(x, boxSize):
    """
    Apply periodic boundary conditions to coordinate x.

    Parameters:
    - x (float): The coordinate value.
    - boxSize (float): The size of the simulation box.

    Returns:
    - x_wrapped (float): The coordinate wrapped into the box.
    """
    return x - boxSize * np.round(x / boxSize)

@numba.njit
def realSpace(positions: np.ndarray, charges: np.ndarray, boxSize: float, alpha: float) -> np.float64:
    """
    Compute the real-space contribution to the Ewald sum.

    Parameters:
    - positions (np.ndarray): Array of particle positions.
    - charges (np.ndarray): Array of particle charges.
    - boxSize (float): Size of the simulation box.
    - alpha (float): Ewald damping parameter.

    Returns:
    - energy (float): The real-space energy contribution per particle.
    """
    # Initialize energy
    energy = 0.0

    n = positions.shape[0]
    quarterBox = 0.25 * boxSize**2  # Square of half the box size

    # Loop over all unique pairs of particles
    for i in range(n - 1):
        for j in range(i + 1, n):
            # Compute displacement vector with periodic boundary conditions
            dr = positions[i] - positions[j]
            dr = wrap(dr, boxSize)
            r2 = np.dot(dr, dr)

            # Apply cutoff to reduce computation
            if r2 < quarterBox:
                r = np.sqrt(r2)
                # Accumulate the pairwise energy
                energy += charges[i] * charges[j] * erfc(alpha * r) / r

    # Return energy per particle
    return energy / n

def fourierSpace(positions: np.ndarray, charges: np.ndarray, boxSize: float, alpha: float, numberOfWaveVectors: int) -> np.float64:
    """
    Compute the Fourier-space contribution to the Ewald sum.

    Parameters:
    - positions (np.ndarray): Array of particle positions.
    - charges (np.ndarray): Array of particle charges.
    - boxSize (float): Size of the simulation box.
    - alpha (float): Ewald damping parameter.
    - numberOfWaveVectors (int): Number of wave vectors in each direction.

    Returns:
    - energy (float): The Fourier-space energy contribution per particle.
    """
    # Initialize energy
    energy = 0.0
    numberOfAtoms = positions.shape[0]

    # Precompute exponentials for e^(i k · r)
    # eik[k, i, d] stores e^(i k_d * r_i^d) for k = 0..numberOfWaveVectors, i = 0..N-1, d = 0..2
    eik = np.ones((numberOfWaveVectors + 1, numberOfAtoms, 3), dtype=np.complex128)
    eik_xy = np.ones(numberOfAtoms, dtype=np.complex128)  # Temporary array for combined x and y components

    # Unit of reciprocal space (k-space)
    boxUnitCircle = 2.0 * np.pi / boxSize
    volume = boxSize ** 3

    # Compute s = (2π / L) * positions
    s = boxUnitCircle * positions

    # Compute eik[1] = e^(i * s)
    eik[1] = np.exp(1j * s)

    # Compute higher order exponentials using recursion relation
    for k in range(2, numberOfWaveVectors + 1):
        eik[k] = eik[k - 1] * eik[1]

    # Loop over wave vectors in the x-direction from 0 to numberOfWaveVectors
    for kx in range(numberOfWaveVectors + 1):
        # Determine the prefactor (accounts for kx = 0 term counted only once)
        prefactor = (1.0 if kx == 0 else 2.0)

        # Loop over wave vectors in the y-direction from -numberOfWaveVectors to numberOfWaveVectors
        for ky in range(-numberOfWaveVectors, numberOfWaveVectors + 1):
            # Get the exponentials for ky
            # Use symmetry: e^(i*(-k)*r) = [e^(i*k*r)]^*
            if ky == 0:
                eiky_temp = eik[0, :, 1]
            elif ky > 0:
                eiky_temp = eik[ky, :, 1]
            else:  # ky < 0
                eiky_temp = np.conj(eik[-ky, :, 1])

            # Combine exponentials in x and y directions
            eik_xy = eik[kx, :, 0] * eiky_temp

            # Loop over wave vectors in the z-direction from -numberOfWaveVectors to numberOfWaveVectors
            for kz in range(-numberOfWaveVectors, numberOfWaveVectors + 1):
                # Skip the zero vector to avoid division by zero
                if kx == 0 and ky == 0 and kz == 0:
                    continue

                # Form the k-vector
                kvec = np.array([kx, ky, kz])
                ksq = np.dot(kvec, kvec)

                # Compute the squared magnitude of the reciprocal vector
                rkvec = boxUnitCircle * kvec
                rksq = np.dot(rkvec, rkvec)

                # Compute the exponential damping factor
                temp = prefactor * np.exp(-0.25 * rksq / alpha ** 2) / rksq

                # Get the exponentials for kz
                # Use symmetry as before
                if kz == 0:
                    eikz_temp = eik[0, :, 2]
                elif kz > 0:
                    eikz_temp = eik[kz, :, 2]
                else:  # kz < 0
                    eikz_temp = np.conj(eik[-kz, :, 2])

                # Compute the structure factor c(k) = Σ_q q_i e^(i k · r_i)
                cksum = np.sum(charges * eik_xy * eikz_temp)

                # Accumulate the energy contribution from this k-vector
                energy += temp * np.abs(cksum) ** 2

    # Multiply by the overall prefactor
    energy *= 2.0 * np.pi / volume

    # Return energy per particle
    return energy / numberOfAtoms

def selfEnergy(charges: np.ndarray, alpha: float):
    """
    Compute the self-energy correction term in the Ewald sum.

    Parameters:
    - charges (np.ndarray): Array of particle charges.
    - alpha (float): Ewald damping parameter.

    Returns:
    - energy (float): The self-energy correction per particle.
    """
    # Compute the self-energy correction
    return -alpha * np.sum(charges ** 2) / (np.sqrt(np.pi) * charges.shape[0])

def totalEnergy(positions: np.ndarray, charges: np.ndarray, boxSize: float, alpha: float, numberOfWaveVectors: int) -> np.float64:
    """
    Compute the total electrostatic energy using Ewald summation.

    Parameters:
    - positions (np.ndarray): Array of particle positions.
    - charges (np.ndarray): Array of particle charges.
    - boxSize (float): Size of the simulation box.
    - alpha (float): Ewald damping parameter.
    - numberOfWaveVectors (int): Number of wave vectors in each direction for the Fourier-space sum.

    Returns:
    - total_energy (float): The total electrostatic energy per particle.
    """
    # Compute real-space contribution
    real = realSpace(positions, charges, boxSize, alpha)

    # Compute Fourier-space contribution
    fourier = fourierSpace(positions, charges, boxSize, alpha, numberOfWaveVectors)

    # Compute self-energy correction
    self = selfEnergy(charges, alpha)

    # Return the total energy
    return real + fourier + self


In [6]:
alpha = 1.0
numberOfWaveVectors = 32
boxSize = 20
charge = 1.0

positions, charges = getPositionsCharges(boxSize, charge)

In [7]:
totalEnergy(positions, charges, boxSize, alpha, numberOfWaveVectors)

-0.8737822973162099